
# ** Naive Bayes Classifier**



 we are going explore one new classification technique:

  - Naive Bayes

We are reusing the version of the Melbourne housing data to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

In addition to building our own Naive Bayes classifier, we are going to compare the performace of our classifier to the [Gaussian Naive Bayes Classifier](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) available in the scikit-learn library.


In [ ]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
from sklearn.naive_bayes import GaussianNB # The only thing in scikit-learn you can use this assignment

# Starting off loading a training set and setting a variable for the target column, "Type"
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')
target_col = 'Type'

## **Q1 - Fix a column of data to be numeric**
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formating that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column.

In [ ]:
# Normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format

def normalize_date(d):
  (day,month,year) = d.split('/')
  if len(year) == 2:
      year = "20" + year
  return( day + "/" + month + "/" + year)


In [ ]:
df_melb['Date'] = df_melb['Date'].apply(normalize_date)
df_melb['unixtime'] = pd.to_datetime(df_melb['Date'], format='%d/%m/%Y').astype(int) // 10**9
df_melb = df_melb.drop(columns="Date")

print("The min unixtime is {:d} and the max unixtime is {:d}".format(df_melb['unixtime'].min(), df_melb['unixtime'].max()))


KeyError: ignored

## **Q2 Calculating the prior probabilities**
Calculate the prior probabilities for each possible "Type" in `df_melb` and populate a dictionary, `dict_priors`, where the key is the possible "Type" values and the value is the prior probabilities. Show the dictionary. Do not hardcode the possible values of "Type".  Don't forget about [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html).

In [ ]:
dict_priors = df_melb['Type'].value_counts(normalize=True).to_dict()


# print the priors
dict_priors

{'h': 0.452, 'u': 0.418, 't': 0.13}

## **Q3 Create a model for the distribution of all of the numeric attributes**
For each class, and for each attribute calculate the sample mean and sample standard deviation.  You should store the model in a nested dictionary, `dict_nb_model`, such that `dict_nb_model['h']['Rooms']` is a tuple containing the mean and standard deviation for the target Type 'h' and the attribute 'Rooms'.  Show the model using the `display` function. You should ignore entries that are `NaN` in the mean and [standard deviation](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html) calculation.

In [ ]:
df_melb.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,2,h,399000,8.7,3032,1,1.0,904,53.0,1985.0,1.462579e+09
1,3,h,1241000,13.9,3165,1,1.0,643,NaN,NaN,1.472342e+09
2,2,u,550000,3.0,3067,1,1.0,1521,NaN,NaN,1.499472e+09
3,3,u,691000,8.4,3072,1,1.0,170,NaN,NaN,1.498262e+09
4,2,u,657500,4.6,3122,1,1.0,728,73.0,1965.0,1.479514e+09


In [ ]:
dict_nb_model = dict()
for target in dict_priors.keys():
    dict_nb_model[target] = dict()
    for attribute in df_melb.columns:
        if df_melb[attribute].dtype in ['int64', 'float64']:
            mean_val = df_melb[df_melb['Type'] == target][attribute].mean()
            std_val = df_melb[df_melb['Type'] == target][attribute].std()

            if not (pd.isnull(mean_val) or pd.isnull(std_val)):
                dict_nb_model[target][attribute] = (mean_val, std_val)

display(dict_nb_model)

{'h': {'Rooms': (3.269911504424779, 0.725826420112775),
  'Price': (1189022.3451327435, 586296.5794417894),
  'Distance': (12.086725663716816, 7.397501132737295),
  'Postcode': (3103.8982300884954, 98.35750345419703),
  'Bathroom': (1.5619469026548674, 0.6720871086493074),
  'Car': (1.7777777777777777, 0.932759177140425),
  'Landsize': (932.9646017699115, 3830.7934157687173),
  'BuildingArea': (156.2433962264151, 54.62662837301433),
  'YearBuilt': (1954.900826446281, 32.4618763471547),
  'unixtime': (1485717578.761062, 13838562.05060146)},
 'u': {'Rooms': (2.0430622009569377, 0.5908453859944255),
  'Price': (634207.1770334928, 217947.32866736987),
  'Distance': (8.760287081339714, 5.609778714430756),
  'Postcode': (3120.4545454545455, 87.18475679946476),
  'Bathroom': (1.1818181818181819, 0.4222815154866222),
  'Car': (1.1483253588516746, 0.47231993860297056),
  'Landsize': (436.23444976076553, 1394.3403794653257),
  'BuildingArea': (83.85585585585585, 45.95943801516662),
  'YearBuilt'

In [ ]:
display(dict_nb_model)

{'h': {'Rooms': (3.269911504424779, 0.725826420112775),
  'Price': (1189022.3451327435, 586296.5794417894),
  'Distance': (12.086725663716816, 7.397501132737295),
  'Postcode': (3103.8982300884954, 98.35750345419703),
  'Bathroom': (1.5619469026548674, 0.6720871086493074),
  'Car': (1.7777777777777777, 0.932759177140425),
  'Landsize': (932.9646017699115, 3830.7934157687173),
  'BuildingArea': (156.2433962264151, 54.62662837301433),
  'YearBuilt': (1954.900826446281, 32.4618763471547),
  'unixtime': (1485717578.761062, 13838562.05060146)},
 'u': {'Rooms': (2.0430622009569377, 0.5908453859944255),
  'Price': (634207.1770334928, 217947.32866736987),
  'Distance': (8.760287081339714, 5.609778714430756),
  'Postcode': (3120.4545454545455, 87.18475679946476),
  'Bathroom': (1.1818181818181819, 0.4222815154866222),
  'Car': (1.1483253588516746, 0.47231993860297056),
  'Landsize': (436.23444976076553, 1394.3403794653257),
  'BuildingArea': (83.85585585585585, 45.95943801516662),
  'YearBuilt'

## **Q4 Write a function that calculates the probability of a Gaussian**

Given the mean ($\mu$), standard deviation ($\sigma$), and a observed point, `x`, return the probability.  
Use the formula $p(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ ([wiki](https://en.wikipedia.org/wiki/Normal_distribution)).  You should use [numpy's exp](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) function in your solution.

In [ ]:
def get_p( mu, sigma, x):
    exponent = -0.5 * ((x - mu) / sigma) ** 2
    p = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(exponent)
    return p

In [ ]:
# Test it
p = get_p( 0, 2, 0.5)
p

0.19333405840142462


## **Q5 Write the Naive Bayes classifier function**
The Naive Bayes classifier function, `nb_class`, should take as a parameter the prior probability dictionary. `dict_priors`, the dictionary containing all of the gaussian distribution information for each attribue, `dict_nb_model`, and a single observation row (a series generated from iterrows) of the test dataframe. It should return a single target classification. For this problem, all of our attributes are numeric and modeled as Gaussians, so we don't worry about categorical data. Make sure to skip attributes that do not have a value in the observation.  Do not hardcode the possible classification types.



In [ ]:
def nb_class( dict_priors, dict_nb_model, observation):
    class_probabilities = {}
    for target, prior in dict_priors.items():
        p = 1
        for attribute, value in observation.items():
            if attribute in dict_nb_model[target]:
                mean, std = dict_nb_model[target][attribute]
                if not np.isnan(value) and std != 0:
                    p *= get_p(mean, std, value)
        class_probabilities[target] = p * prior
    return max(class_probabilities, key=class_probabilities.get)

## **Q6 Calculate the accuracy using Naive Bayes classifier function on the test set**
Load the test set from file, convert date to unix time and drop the date column, classify each row using your `nb_class`, and then show the accuracy on the test set.

In [ ]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')
df_test['Date'] = df_test['Date'].apply( normalize_date )
df_test['unixtime'] =  df_test['Date']
df_test = df_test.drop(columns="Date")

In [ ]:
predictions = []

for (indx, row) in df_test.iterrows():
    observation = row.drop('Type')
    predicted_class = nb_class(dict_priors, dict_nb_model, observation)
    predictions.append(predicted_class)

TypeError: ignored

In [ ]:
correct_predictions = sum(predictions == df_test['Type'])
total_predictions = len(df_test)
acc = correct_predictions / total_predictions

ValueError: ignored

In [ ]:
print('Accuracy is {:.2f}%'.format(acc * 100))

NameError: ignored

### **Q7 Use `scikit-learn` to do the same thing**

Now we understand the inner workings of the Naive Bayes algorithm, let's compare our results to [scikit-learn's Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) implementation. Use the [GaussianNB](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) to train using the `df_melb`dataframe and test using the `df_test` dataframe. Remember to split `df_melb` into a `df_X` with the numerical attributes, and a `s_y` with the target column. On the `df_melb` frame you will have to fill the empty attributes via imputation since the scikit-learn library can not handle missing values.  Use the same method you used in the last homework (filling the training data with the mean of the non-nan values).

In [ ]:
# Imputation training
dict_imputation = {}
for col in df_melb.columns:
    if df_melb[col].dtype != "object":
        imputer = SimpleImputer(strategy='mean')
        df_melb[col] = imputer.fit_transform(df_melb[[col]])
        dict_imputation[col] = imputer

# Imputation - apply on the test data
for col, imputer in dict_imputation.items():
    df_test[col] = imputer.transform(df_test[[col]])

# Seperate the attributes from the target_col
s_test = df_test['Type']
df_test = df_test.drop(columns=['Type'])

df_X = df_melb.drop(columns=['Type'])
s_y = df_melb['Type']

NameError: ignored

In [ ]:
gnb = GaussianNB()
y_pred = gnb.predict(df_test)

NotFittedError: ignored

In [ ]:
acc = (y_pred == s_test).mean()
print('Accuracy is {:.2f}%'.format(acc*100))

NameError: ignored